In [1]:
import sys
sys.path.append("/home/ly/workspace/mmsa")
seed = 13231
import numpy as np
import torch
from torch import nn
from torch import optim

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
from models.vistanet_test import *
from utils.train import *
from typing import *
from collections import Counter

from utils.load_yelp import *
from utils.dataset import *
from utils.train import *

In [2]:
config = default_config
config

{'task': 'CLS',
 'embedding_dim': 200,
 'embedding': None,
 'freeze_embedding': False,
 'word_hidden_size': 50,
 'word_layers': 1,
 'word_proj_hidden_size': 100,
 'uniform_bound': 0.1,
 'sentence_hidden_size': 50,
 'sentence_layers': 1,
 'sent_proj_hidden_size': 100,
 'img_input_size': 4096,
 'visual_attention_size': 100,
 'dropout': 0.5,
 'output_size': 5,
 'bias_init': 1.0}

In [3]:
train_set, valid_set, test_set = load_glove_vgg_data(config)

In [4]:
batch_size = 128
workers = 6
train_loader, valid_loader, test_loader = get_loader(batch_size, workers, get_collate_fn(config), train_set, valid_set, test_set)

In [5]:
model = Model(config).cuda()
loss = nn.CrossEntropyLoss()
get_parameter_number(model), loss

/home/ly/miniconda3/envs/torch1.6.0/lib/python3.7/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


({'Total': 9128805, 'Trainable': 9128805}, CrossEntropyLoss())

In [6]:
%%time
y_true, y_pred, _loss = predict(model, valid_loader, loss)
evalute(y_true, y_pred), Counter(y_pred)

CPU times: user 3.5 s, sys: 670 ms, total: 4.17 s
Wall time: 4.77 s


((0.18600451467268622, 0.07581024399744264), Counter({4: 4239, 3: 127, 2: 64}))

In [7]:
viz = get_Visdom()
_interval = 5
batch_loss_drawer = VisdomScalar(viz, f"batch_loss interval:{_interval}")
epoch_loss_drawer = VisdomScalar(viz, f"Train and valid loss", 2)
acc_drawer = VisdomScalar(viz, "Train and valid accuracy", 2)

batch_loss = []
train_loss = []
valid_loss = []
train_acc = []
valid_acc = []


Setting up a new session...


In [8]:
%%time
lr = 1e-3
epoches = 20
optimizer = get_regal_optimizer(model, optim.AdamW, lr)
res, model = train_visdom(model, optimizer, loss, viz, train_loader,
                          valid_loader, epoches, batch_loss, batch_loss_drawer,
                          train_loss, valid_loss, epoch_loss_drawer,
                          train_acc, valid_acc, acc_drawer,
                         _interval=_interval)

No 1 epoch:: 277it [01:57,  2.36it/s]


No 1 epoch: train_loss:1.354967823484617 train_acc:0.37127944703061083 valid_loss:1.0435471039593354 valid_acc:0.5573363431151241


No 2 epoch:: 277it [01:56,  2.38it/s]


No 2 epoch: train_loss:1.2089249277740426 train_acc:0.4372125828748766 valid_loss:0.9804539188305358 valid_acc:0.5959367945823928


No 3 epoch:: 277it [01:58,  2.35it/s]


...

No 6 epoch: train_loss:0.9524385685072512 train_acc:0.5469318662716885 valid_loss:0.982568848321454 valid_acc:0.5747178329571107


No 7 epoch:: 277it [01:58,  2.34it/s]


No 7 epoch: train_loss:0.8854872923809444 train_acc:0.5784736916349273 valid_loss:1.020452000325205 valid_acc:0.5751693002257336


No 8 epoch:: 277it [01:57,  2.35it/s]


No 8 epoch: train_loss:0.8210435211111955 train_acc:0.6028494851177881 valid_loss:1.1132246146621876 valid_acc:0.5699774266365688
CPU times: user 14min 49s, sys: 1min 22s, total: 16min 12s
Wall time: 16min 17s


In [15]:
res

{'max_acc': 0.609255079006772,
 'max_acc_epoch': 3,
 'max_train_acc': 0.4634222034137396,
 'max_acc_train_loss': 1.1434450638211822,
 'max_acc_valid_loss': 0.9291869199841071,
 'last_acc': 0.5699774266365688,
 'last_train_acc': 0.6028494851177881,
 'last_epoch': 8,
 'last_train_loss': 0.8210435211111955,
 'last_valid_loss': 1.1132246146621876}

In [16]:
eval_model(model, test_loader, loss)

((0.6060948081264108, 0.602188563002813), 0.9252480728631634)

In [17]:
%load_ext line_profiler

In [18]:
X, y = next(iter(valid_loader))
X, y = to_cuda((X, y))
model.eval()

Model(
  (word_embedding): Embedding(42834, 200)
  (word_encoder): AttentionDynamicRNN(
    (rnn): DynamicRNN(
      (RNN): GRU(200, 50, batch_first=True, dropout=0.5, bidirectional=True)
    )
    (proj_fc): Linear(in_features=100, out_features=100, bias=True)
    (tanh): Tanh()
    (mask_layer): MaskLayer()
    (softmax): Softmax(dim=1)
  )
  (padding_layer): PaddingLayer()
  (sent_encoder): DynamicRNN(
    (RNN): GRU(100, 50, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (sent_proj_fc): Linear(in_features=100, out_features=100, bias=True)
  (sent_proj_tanh): Tanh()
  (img_proj_fc): Linear(in_features=4096, out_features=100, bias=True)
  (img_proj_tanh): Tanh()
  (visual_softmax): Softmax(dim=2)
  (doc_proj_fc): Linear(in_features=100, out_features=100, bias=True)
  (doc_proj_tanh): Tanh()
  (doc_visual_softmax): Softmax(dim=1)
  (output_layer): OutputLayer(
    (fc): Linear(in_features=100, out_features=5, bias=True)
    (dropout): Dropout(p=0.5, inplace=True)
  )
)

In [19]:
%lprun -f model.forward model.forward(X)

Timer unit: 1e-06 s

Total time: 0.131673 s
File: /mnt/disk1/ly/workspace/mmsa/models/vistanet_test.py
Function: forward at line 63

Line #      Hits         Time  Per Hit   % Time  Line Contents
    63                                               def forward(self, X):
    64         1          4.0      4.0      0.0          padded_text, lens, sents_num, imgs = X
    65                                                   # 句子编码
    66         1        633.0    633.0      0.5          emb = self.word_embedding(padded_text)
    67         1     108474.0 108474.0     82.4          si = self.word_encoder(emb, lens)
    68         1      11679.0  11679.0      8.9          padded_si = self.padding_layer(si, sents_num)
    69                                                   # 文档编码
    70         1       3794.0   3794.0      2.9          hi, _ = self.sent_encoder(padded_si, sents_num) # gru
    71         1          3.0      3.0      0.0          b, seq, d = hi.shape
    72                    